# **Recurrent Neural Network - RNN**

Les réseaux de neurones récurrents sont relativement récents et très utilisés, notamment dans l'analyse textuelle mais pas seulement.

## **1. L'idée derrière les RNN**

### **1.1 Intro**

Pour l'instant, on a vu :
- les réseaux ANN utilisés pour régression et classification
- les réseaux CNN utilisés pour la computer vision

**Place donc aux RNN !**

Vidéos d'intro :
- aux RNN : https://www.youtube.com/watch?v=EL439RMv3Xc
- aux LSTM : https://www.youtube.com/watch?v=3xgYxrNyE54
- Sunsprising : film écrit par un RNN https://arstechnica.com/gaming/2016/06/an-ai-wrote-this-movie-and-its-strangely-moving/

Un peu de lecture :
- https://blog.octo.com/les-reseaux-de-neurones-recurrents-des-rnn-simples-aux-lstm/

### **1.2 Lien avec le cerveau humain**

Les poids sont attachés aux synapses et représentent notre capacité à "apprendre" en ajustant les poids.

Dans un réseau comme on a vu jusqu'à maintenant, les poids correspondent en fait à notre mémoire à long terme : si on laisse le réseau et qu'on revient plus tard, la valeur des poids sera toujours là.

Les réseaux récurrents sont faits pour représenter la mémoire à court terme afin de prédire les valeurs futures.

### **1.3 Représentation des RNN**

Afin de représenter les RNN on va effectuer les "transformations" suivantes :

1. On applatit le réseau et on réduit toutes les connections à une seule flèche puis on le met au format vertical. Bien comprendre que dans les 3ème et 4ème versions, chaque cellule représente en fait une couche d'un réseau avec tous les neurones qui la composent et chaque flèche représente une multitude de connections entre tous les neurones de 2 couches successives.

<img src="img/rnn1.png" width=400px> <img src="img/rnn2.png" width=400px> <img src="img/rnn3.png" width=400px> <img src="img/rnn4.png" width=70px>

2. Ensuite on ajoute une flèche récursive sur la couche cachée qui signifie que la sortie de la couche cachée donne un output mais est par ailleurs réintroduite dans la cellule pour les prédictions futures. C'est en fait le principe même du RNN. Cette première version est compacte mais un peu simpliste. On peut pour y voir plus clair ajouter une notion temporelle. Au cours du temps, chaque sortie de la couche caché est réintégrée en tant qu'input à l'instant suivant. C'est là qu'on retrouve le principe de "mémoire à court terme".

<img src="img/rnn5.png" width=100px> <img src="img/rnn6.png" width=600px>

Il y a plusieurs types d'architectures pour les RNN, chacune répondant à une problématique différente. Quelques exemples pour y voir plus clair.

<img src="img/rnn8.png" width=1000px>

- **One-to-many :** une seule entrée et plusieurs sorties par exemple pour la description d'une image avec une syntaxe grammaticalement correcte et pas uniquement une liste de features présents dans l'image
- **Many-to-one :** plusieurs entrées et une seule sortie par exemple pour de l'analyse de sentiment sur des commentaires
- **Many-to-many :** plusieurs entrées et plusieurs sorties :
>- par exemple pour une traduction, on ne peut pas traduire mot à mot mais il faut tenir compte du contexte (accords en genre, en nombre etc...)
>- par exemple pour la description d'une vidéo, on ne peut pas faire juste de l'analyse image par image mais il faut tenir compte de l'image précédente pour déterminer l'action en cours

## **2. La fuite du gradient**

Référence : http://proceedings.mlr.press/v28/pascanu13.pdf

En british : *vanishing gradient*. Le principe c'est que le gradient (dont on se sert pour la mise à jour des poids du réseaux vous le savez déjà) devient tellement faible que la mise à jour des poids ne se fait plus ou extrêmement lentement. On va détailler un peu tout ça.

### **2.1 Le gradient**
Pour rappel, l'objectif de l'algorithme du gradient est la minimisation de notre fonction de coût qui, elle, mesure l'erreur de notre modèle. Ça se passe comme ça :

<img src="img/rnn9.png" width=400px>

Je vous renvoie sur ce qu'on a fait pour la descente de gradient si c'est pas clair ou d'aller faire un tour [là par exemple](https://developers.google.com/machine-learning/crash-course/reducing-loss/video-lecture?hl=fr).
À retenir : le gradient est le signal qui permet de savoir dans quelle direction modifier les poids pour minimiser l'erreur. Par ailleurs, la valeur du gradient intervient dans le calcul des nouveaux poids (modulo un paramètre qui est le taux d'apprentissage, remember ?)

Dans le cadre des réseaux de neurones, l'algorithme du gradient devient ce qu'on appelle la rétropropagation qu'on a détaillé dans la partie sur les ANN :  
propagation de l'input dans le réseau $\rightarrow$ output avec une erreur $\rightarrow$ calcul de cette erreur via la fonction de coût $\rightarrow$ rétropropagation pour trouver le minimum global correspondant à la valeur optimale des poids

### **2.2 Le problème**
Pour les réseaux récurrents, c'est plus compliqué...en effet la propagation avant ne se fait pas linéairement mais récursivement (flèches jaunes ci-dessous) et de la même manière, la rétropropagation ne l'est pas non plus (flèches violettes).  
Notations dans les graphiques ci-dessous: $y_t, \epsilon_t$ output et erreur à l'instant $t$ et $W_{in}, W_{out}, W_{rec}$ les poids des synapses d'input, d'output et de récursivité.  
**/!\ Remarque importante :** ne pas être induit en erreur par cette représentation, les poids $W_{in}, W_{out}, W_{rec}$ sont les mêmes à chaque instant $t$

<img src="img/rnn10.png" width=440px> <img src="img/rnn11.png" width=455px>

Où est le problème ? Sans entrer dans les maths, retenez qu'à chaque fois qu'on remonte à une couche précédente, le gradient est multiplié par $W_{rec}$, qui relie les couches entre elles au cours du temps. Or $W_{rec}$ est généralement faible et lorsqu'on multiple par des valeurs comprises entre 0 et 1, on est de plus en plus petit. À  l'arrivée, on se retrouve avec des valeurs de gradient trop faibles pour pouvoir entraîner les couches les plus éloignées. Et par conséquent comme la sortie dépend de ces couches-là, l'apprentissage du réseau est inefficace voire non convergent...c'est le **vanishing gradient**.

Mathématiquement, on peut le voir dans les formules (extraites de l'article cité en intro).
À l'équation (5), le terme multiplicatif $W_{rec}$ est présent dans le calcul de la dérivée partiel par rapport à l'état du réseau aux instants précédents et on voit qu'il est multiplié autant de fois que l'on remonte.

<img src="img/rnn14.png" width=400px> <img src="img/rnn13.png" width=400px>

Le même problème se pose si $W_{rec}$ prend de grandes valeurs. Cette fois, à chaque étape, on va multiplier par une grande valeur et le gradient va finir par être beaucoup trop grand pour converger (on va tourner autour du minimum sans jamais réussir à l'atteindre). On parle d'**explosion du gradient ou exploding gradient** 

<img src="img/rnn12.png" width=400px>

### **2.3 Les solutions**

Solutions exploding gradient :
1. on tronque la rétropropagation pour éviter mais tous les poids ne sont pas mis à jour
2. pénalisation du gradient pour réduire artificiellement sa valeur
3. gradient clipping qui consiste à fixe un maximum

Solutions vanishing gradient :
1. initialisation des poids de manière à minimiser le problème mais pas évident
2. Echo State Networks une solution dont on parlera pas
3. les fameux **LSTM**

## **3. LSTM : Long Short-Term Memory**

Des références :
- https://colah.github.io/posts/2015-08-Understanding-LSTMs/
- https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- https://arxiv.org/pdf/1506.02078.pdf

### **3.1 Kessecéssa**

Si $W_{rec}>1$, alors on a le problème de l'explosion du gradient.  
Si $W_{rec}<1$, alors on a le problème de la fuite du gradient.  
Du coup, l'idée en gros va être d'avoir $W_{rec}=1$, enfin en un peu plus complexe.

En pratique, le principe du LSTM est d'ajouter un vecteur de "mémoire" de taille fixe pour sélectionner (donc garder ou supprimer) de l'information via de nouveaux poids appliqués à ce vecteur. Plus précisément :

Voilà la représentation d'une cellule d'un RNN simple dans lequel on doit faire face au problème de la fuite du gradient. Le problème vient de l'opération avec $tanh$ dans la cellule. $tanh$ pour [tangeante hyperbolique](https://fr.wikipedia.org/wiki/Tangente_hyperbolique) est une autre fonction d'activation dont la dérivée est comprise entre 0 et 1 (le problème est le même avec la fonction sigmoïde). Cette fonction est intéressante pour les réseaux récurrents car elle est bornée entre -1 et 1 et par conséquent cela évite que les valeurs explose puisqu'on l'utilise de manière récurrente.

<img src="img/rnn15.png" width=700px>

Représentation d'une cellule d'un LSTM :

<img src="img/rnn16.png" width=700px>

Une première remarque fondamentale : la ligne noire en haut qui traverse les celulles les unes après les autres est cette fameuse "mémoire" qu'on a ajoutée et qui correspond à $W_{rec}=1$. On y fait que des opérations linéaires simples (multiplication et addition) et on peut propager et rétropropager l'erreur selon cette ligne sans problème du vanishing gradient.

Petit zoom sur cette cellule pour y voir plus clair.

<img src="img/rnn17.png"  width=400px>

- $C_t$ : mémoire à l'instant $t$
- $h_t$ : information à l'instant $t$ qui va dans le module suivant à $t+1$
- $x_t$ : nouvelle information qui vient dans la cellule, input à l'instant $t$
- $\sigma$ : fonction sigmoïde
- $tanh$ : fonction tangeante hyperbolique
- $\times, +$ : multiplication et somme

Remarques : $C, h, x$ sont des vecteurs qui contiennent donc plusieurs valeurs

Chaque "carré jaune" correspond à une couche d'un réseau.

Chaque combo d'opération "sigmoïde"+"cercle rouge multiplication" correspond à ce qu'on appelle une valve ou *gate*. Le terme est bien choisi car le but est de laisser passer ou non l'information.

__*Forget Gate*__ : capacité à oublier certaines informations jugées inutiles  
= sachant les nouveaux inputs et l'état précédant, on définit quelles vont être les infos à oublier (vecteur de poids $f_t$ à appliquer sur le vecteur mémoire)

<img src="img/rnn18.png"  width=600px>

__*Input Gate*__ : ajout d'informations utiles  
= propose des nouvelles infos à mettre dans le vecteur mémoire (modulé par un nouveau vecteur de poids $i_t$) qu'on ajoute à la mémoire "post-suppression"

<img src="img/rnn19.png"  width=600px>

<img src="img/rnn20.png"  width=552px>

__*Output Gate*__ : état de la cellule à chaque instant selon les infos ajoutées et oubliées  
= sélectionne les infos dans la mémoire (via un nouveau vecteur de poids $o_t$)

<img src="img/rnn21.png"  width=600px>

Remarques : les matrices des poids $W_f, W_i, W_C, W_o$ sont apprises par rétropropagation du gradient. C'est donc le réseau qui pendant l'entraînement apprend quand est-ce que c'est intéressant d'ouvrir/fermer les valves.

### **3.3 Les variantes de LSTM**

Une première variante consiste à permettre aux couches *gate* de regarder l'état de la cellule "mémoire" pour décider quelle information garder ou non.

<img src="img/rnn22.png"  width=600px>

Une seconde variante consiste à connecter les *forgate gate* et *input gate* pour décider conjointement comment garder/supprimer. Elles sont reliées de telle sorte que si on ouvre une gate alors on ferme l'autre.

<img src="img/rnn23.png"  width=600px>

Une troisième variante est l'architecture **Gated Recurrent Unit (GRU)** qui est assez récente. Cette variante consiste à combiner ensemble *forget gate* et *input gate* pour n'avoir qu'une seule *update gate*. Une autre différence et qu'elle n'a pas de cellule mémoire explicite $C_t$ mais elle fusionne l'état caché de la cellule et la cellule mémoire dans $h_t$. Ce modèle est finalement plus simple donc plus rapide à calculer pour des résultats équivalent d'où sa popularité en hausse.

<img src="img/rnn24.png"  width=600px>

### **3.3 Les RNN profonds**

Pour aller plus loin, il y a les réseaux RNN profonds. Deux sortes :
- les réseaux empilés qui empilent des couches :

<img src="img/rnn26.png"  width=250px>

>- chaque couche peut avoir une architecture différente
>- la séquence de sortie de la première couche est la séquence d'entrée de la seconde, etc...(il faudra donc récupérer les sorties pour chaque sous-couche, ça tombe bien, avec `keras`, il y a un paramètre fait pour ça)

- les réseaux bidirectionnels :

<img src="img/rnn25.png"  width=300px>

>- ajout d'un second RNN qui lit la séquence à l'envers
>- permet d'avoir de l'info sur ce qui se passe **avant et après**
>- /!\ 2 réseaux différents donc 2 jeux de paramètres différents 